In [1]:
import sys
sys.path.append('../')

import os
import os.path as osp
import numpy as np
import pickle

from dataloading.imports import GESTURE_NAMES
from dataloading.visualize import visualize_samples
from dataloading.transform import normalize_vectors, center_scale, integrate_acceleration

In [22]:
with open('../data/uwave/uwave.pkl', 'rb') as f:
    data = pickle.load(f)
data = integrate_acceleration(data)
data = center_scale(data, scale_per_axis=True)

count = 0
for gn in data:
    for sample in data[gn]:
        count += np.sum(np.isnan(sample))
print(f'Nan: {count}')

Nan: 0


../dataloading/transform.py:34: RuntimeWarning: invalid value encountered in true_divide
  centered_scaled /= np.abs(np.max(centered_scaled, axis=0))


In [15]:
%matplotlib inline

name, idx = 'counter-clw', 7
num_visualize = 9
gestures = [GESTURE_NAMES[idx] for i in range(num_visualize)]
indices = np.random.randint(500, size=num_visualize)

# visualize_samples([data[gestures[i]][indices[i]] for i in range(num_visualize)], axes=(0, 1), save_path=f'../plots/{name}_IntegScale_01.png', figsize=(8, 8)) 


In [23]:
def polyfit_timeseries(data, deg=3):
    data_poly = {gesture_name: [] for gesture_name in data}
    for gesture_name in data:
        for i_sample, sample in enumerate(data[gesture_name]):
            poly = np.polynomial.polynomial.polyfit(np.arange(len(sample)), sample, deg=3)
            data_poly[gesture_name].append(poly.T.flatten()) # [x0, x1, x2, x3, y0, y1, y2, y3, z0, z1, z2, z3]

    return data_poly

def aggregate_data(data):
    num_samples = np.sum([len(data[g]) for g in GESTURE_NAMES])
    dim = len(data[GESTURE_NAMES[0]][0])
    X = np.zeros((num_samples, dim), dtype=np.float)
    y = np.zeros(num_samples, dtype=np.int)
    
    idx = 0
    for i_class, gesture_name in enumerate(GESTURE_NAMES):
        for sample in data[gesture_name]:
            X[idx, :] = sample
            y[idx] = i_class
            idx += 1

    return X, y

def normalize_X(X, mean=None, std=None):
    if mean is None and std is None:
        mean = np.mean(X, axis=0)
        std = np.std(X)
        
    X_normed = (X - mean) / std
    return X_normed, mean, std

def random_split(X, y, train_fraction=0.7):
    assert len(X) == len(y)
    test_fraction = 1.0 - train_fraction
    indices_test = []
    
    # Sample the same fraction from each class to prevent (further) imbalance
    for i_class in np.unique(y):
        possible_indices = np.argwhere(y == i_class).flatten()
        selected_indices = np.random.choice(possible_indices, size=int(test_fraction*len(possible_indices)), replace=False) # Select test indices to require less random samples
        indices_test.extend(list(selected_indices))

    # Get the "inverse" indices for training
    slice_train = np.ones(len(X), dtype=np.bool)
    slice_train[indices_test] = False
    
    return X[slice_train], y[slice_train], X[indices_test], y[indices_test]


data_poly = polyfit_timeseries(data)
X, y = aggregate_data(data_poly)
print(X.shape, y.shape)

X_train, y_train, X_test, y_test = random_split(X, y)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

/home/imanox/.local/lib/python3.6/site-packages/numpy/polynomial/polynomial.py:1350: RankWarning: The fit may be poorly conditioned
  return pu._fit(polyvander, x, y, deg, rcond, full, w)


(4481, 12) (4481,)
(3137, 12) (3137,) (1344, 12) (1344,)


In [26]:
'''
TODO
- integrate 0/1/2
- normalize vectors
- center&scale
- poly degrees
- which axes
- normalize poly-coefs to zero-mean and unit variance
'''

from sklearn.linear_model import LogisticRegression

with open('../data/uwave/uwave.pkl', 'rb') as f:
    data = pickle.load(f)

data = integrate_acceleration(data)
data = center_scale(data, scale_per_axis=True)  

data_poly = polyfit_timeseries(data)

X, y = aggregate_data(data_poly)
X_train, y_train, X_test, y_test = random_split(X, y)


classifier = LogisticRegression(max_iter=1000).fit(X_train, y_train)
pred_train = classifier.predict(X_train)
pred_test = classifier.predict(X_test)

train_acc = np.mean(pred_train == y_train)
test_acc = np.mean(pred_test == y_test)

print(f'Acc: {train_acc:0.2f} / {test_acc:0.2f}')

../dataloading/transform.py:34: RuntimeWarning: invalid value encountered in true_divide
  centered_scaled /= np.abs(np.max(centered_scaled, axis=0))
/home/imanox/.local/lib/python3.6/site-packages/numpy/polynomial/polynomial.py:1350: RankWarning: The fit may be poorly conditioned
  return pu._fit(polyvander, x, y, deg, rcond, full, w)


Acc: 0.69 / 0.70
